In [4]:
import xml.etree.ElementTree as ET 
import pandas as pd
import os
import csv

master = pd.read_csv(os.getcwd() + "/Resources/RugbyHub_master_data.csv",dtype=str)

#PLID,TEAMIDのインポート
psheets = ["PLID", "TID", "Venue"]
ptid = pd.read_excel(os.getcwd() +"/Resources/Japan Rugby League One Player ID.xlsx", sheet_name = psheets,dtype=str)
tid = pd.DataFrame(ptid["TID"])
venue = pd.DataFrame(ptid["Venue"])
plid = pd.DataFrame(ptid["PLID"])

# XMLファイルを解析
tree = ET.parse('/Users/isakakou/Desktop/933693_NECGvKINT_final_whistle_superscout.xml') 

# XMLを取得
root = tree.getroot()
df = pd.DataFrame(index=[],columns = ['ID','FXID','PLID','team_id','ps_timestamp','ps_endstamp','MatchTime','psID','period',
           'x_coord','y_coord','x_coord_end','y_coord_end','action','ActionType','Actionresult',
           'qualifier3','qualifier4','qualifier5','Metres','PlayNum','SetNum',
           'sequence_id','player_advantage','score_advantage','flag','advantage','assoc_player'])

for action in root.iter('ActionRow'):
    s = pd.Series([action.get('ID'),action.get('FXID'),action.get('PLID'),action.get('team_id'),
                   action.get('ps_timestamp'),action.get('ps_endstamp'),action.get('MatchTime'),action.get('psID'),
                   action.get('period'),action.get('x_coord'),action.get('y_coord'),action.get('x_coord_end'),action.get('y_coord_end'),
                   action.get('action'),action.get('ActionType'),action.get('Actionresult'),action.get('qualifier3'),
                   action.get('qualifier4'),action.get('qualifier5'),action.get('Metres'),action.get('PlayNum'),action.get('SetNum'),
                   action.get('sequence_id'),action.get('player_advantage'),action.get('score_advantage'),action.get('flag'),
                   action.get('advantage'),action.get('assoc_player')], index = df.columns)
    df = df.append(s, ignore_index=True)
print(df)    
df.qualifier3 = df.qualifier3.map(master.set_index('ID').Definition)
df.qualifier4 = df.qualifier4.map(master.set_index('ID').Definition)
df.qualifier5 = df.qualifier5.map(master.set_index('ID').Definition)
df.Actionresult = df.Actionresult.map(master.set_index('ID').Definition)
df.ActionType = df.ActionType.map(master.set_index('ID').Definition)
df.action = df.action.map(master.set_index('ID').Definition)

df.team_id = df.team_id.map(tid.set_index('team_id').team_name)
df.PLID = df.PLID.map(plid.set_index('players_id').player_known_name)
print(df)
df.to_csv('/Users/isakakou/Desktop/933693_NECGvKINT_final_whistle_superscout.csv', header=True, index=False)

        ID    FXID   PLID team_id ps_timestamp ps_endstamp MatchTime psID  \
0        0  933693   5206    5206         0.00        0.00         2    0   
1        1  933693  21232   19568         0.00        0.00         2    0   
2        2  933693  21232   19568         0.00        0.00         2    0   
3        3  933693  19568   19568         0.00        3.96         6    0   
4        4  933693   5206    5206        50.56       50.56        53    0   
...    ...     ...    ...     ...          ...         ...       ...  ...   
1726  1726  933693  25870   19568      6137.08     6137.08      8234    0   
1727  1727  933693  19568   19568      6138.64     6142.84      8236    0   
1728  1728  933693  26655   19568      6139.80     6139.80      8237    0   
1729  1729  933693  19568   19568      5901.16     6142.84      8240    0   
1730  1730  933693  19568   19568      5901.16     6142.84      8240    0   

     period x_coord  ... qualifier5 Metres PlayNum SetNum sequence_id  \
0 